In [1]:
import cv2
import mediapipe as mp

In [2]:
class handDetector():
    def __init__ (self,mode=False,maxHands=2,detectionCon=0.5,trackCon=0.5):
        self.mode=mode 
        self.maxHands=maxHands
        self.detectionCon=detectionCon
        self.trackCon=trackCon
        self.mpHands=mp.solutions.hands
        self.hands=self.mpHands.Hands(self.mode,self.maxHands,self.detectionCon,self.trackCon) 
        self.mpDraw=mp.solutions.drawing_utils
        self.tipIds=[4,8,12,16,20]
        
    def findHands(self,img,draw=True):
        imgRGB=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        self.results=self.hands.process(imgRGB)
        if self.results.multi_hand_landmarks:
            for hand_landmarks in self.results.multi_hand_landmarks:
                if draw:
                    self.mpDraw.draw_landmarks(img,hand_landmarks,self.mpHands.HAND_CONNECTIONS)
        return img    
    def findPosition(self,img,handno=0,draw=True):
#         xList=[]
#         yList=[]
#         bbox=[]
        self.landmarks_list=[]
        if self.results.multi_hand_landmarks:
            myHand=self.results.multi_hand_landmarks[handno]
            for id,lm in enumerate(myHand.landmark):
                h,w,c=img.shape
                cx,cy=int(lm.x*w),int(lm.y*h)
#                 xList.append(cx)
#                 yList.append(cy)
                self.landmarks_list.append([id,cx,cy])
                if draw:
                    cv2.circle(img,(cx,cy),5,(255,0,255),cv2.FILLED)
#             xmin,xmax=min(xList),max(xList)
#             ymin,ymax=min(yList),max(yList)
#             bbox=xmin,ymin,xmax,ymax
            
            if draw:
                cv2.rectangle(img,(bbox[0]-20,bbox[1]-20),(bbox[2]+20,bbox[3]+20),(0,255,0),2)
        return self.landmarks_list
    def fingersUp(self):
        fingers = []#storing final result
        # Thumb < sign only when  we use flip function to avoid mirror inversion else > sign
        if self.landmarks_list[self.tipIds[0]][1] > self.landmarks_list[self.tipIds[0] - 1][1]:#checking x position of 4 is in right to x position of 3
            fingers.append(1)
        else:
            fingers.append(0)

        # Fingers
        for id in range(1, 5):#checking tip point is below tippoint-2 (only in Y direction)
            if self.landmarks_list[self.tipIds[id]][2] < self.landmarks_list[self.tipIds[id] - 2][2]:
                fingers.append(1)
            else:
                fingers.append(0)

#             totalFingers = fingers.count(1)

        return fingers

In [7]:
detector=handDetector()
cap=cv2.VideoCapture(0)
while True:
    ret,img=cap.read()
    if not ret:
        continue
    img=detector.findHands(img)
    landmarks_list=detector.findPosition(img,draw=False)
    if(len(landmarks_list)!=0):
        fingers=detector.fingersUp()
        if(fingers.count(1)==0):
            cv2.putText(img,'0',(30,30),1,2,(0,255,0),5,cv2.LINE_AA)
        elif(fingers.count(1)==1 and fingers[1]==1):
            cv2.putText(img,'1',(30,30),1,2,(0,255,0),5,cv2.LINE_AA)
        elif(fingers.count(1)==2 and fingers[1]==1 and fingers[2]==1) :
            cv2.putText(img,'2',(30,30),1,2,(0,255,0),5,cv2.LINE_AA)
        elif(fingers.count(1)==3 and fingers[0]==1 and fingers[1]==1 and fingers[2]==1):
            cv2.putText(img,'3',(30,30),1,2,(0,255,0),5,cv2.LINE_AA)
        elif(fingers.count(1)==4 and fingers[0]==0) :
            cv2.putText(img,'4',(30,30),1,2,(0,255,0),5,cv2.LINE_AA)
        elif(fingers.count(1)==5)    :
            cv2.putText(img,'5',(30,30),1,2,(0,255,0),5,cv2.LINE_AA)
        elif(fingers.count(1)==3 and fingers[0]==0 and fingers[4]==0) :
            cv2.putText(img,'6',(30,30),1,2,(0,255,0),5,cv2.LINE_AA)
        elif(fingers.count(1)==3 and fingers[0]==0 and fingers[3]==0) :
            cv2.putText(img,'7',(30,30),1,2,(0,255,0),5,cv2.LINE_AA)  
        elif(fingers.count(1)==3 and fingers[0]==0 and fingers[2]==0) :
            cv2.putText(img,'8',(30,30),1,2,(0,255,0),5,cv2.LINE_AA)  
        elif(fingers.count(1)==3 and fingers[0]==0 and fingers[1]==0) :
            cv2.putText(img,'9',(30,30),1,2,(0,255,0),5,cv2.LINE_AA) 
        else:
            cv2.putText(img,'nothing',(30,30),1,2,(0,255,0),5,cv2.LINE_AA)
    else:  
        cv2.putText(img,'nothing',(30,30),1,2,(0,255,0),5,cv2.LINE_AA)
    cv2.imshow('image',img)
    k=cv2.waitKey(1)
    if(k==27):
        break
cap.release()
cv2.destroyAllWindows()